In [1]:
import pandas as pd
import sqlite3
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

In [2]:
# viewing data folder

! ls ../unzippedData

bom.movie_gross.csv  rt.movie_info.tsv    tmdb.movies.csv
im.db                rt.reviews.tsv       tn.movie_budgets.csv


In [3]:
# making connection to imdb

conn = sqlite3.connect('../unzippedData/im.db')

In [4]:
MovieBudget = pd.read_csv('../unzippedData/tn.movie_budgets.csv')
# MovieBudget

In [5]:
# function for cleaning numbers column

def clean_number_series(Series):
    Series = Series.str.replace(",", "")
    Series = Series.str.replace("$", "")
    Series = Series.astype(int)
    return Series

# function for cleaning Column headers

def clean_column_headers(dataframe):
    dataframe.columns = dataframe.columns.str.title()
    dataframe.columns = dataframe.columns.str.replace("_", " ")
    
# function for cleaning comma seperated string values of a series into list of strings. Accepts one series.

def clean_comma_series(Series):
    Series = Series.str.replace("_", " ")
    Series = Series.str.title()
    Series = Series.str.split(",")
    return Series

In [6]:
# Reformatted Movie Budget

clean_column_headers(MovieBudget)
MovieBudget['Worldwide Gross'] = clean_number_series(MovieBudget['Worldwide Gross'])
MovieBudget['Domestic Gross'] = clean_number_series(MovieBudget['Domestic Gross'])
MovieBudget['Production Budget'] = clean_number_series(MovieBudget['Production Budget'])


/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_952/1531069200.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Series = Series.str.replace("$", "")


In [7]:
# Making Profitability Column and Revenue Budget Ratio Column

MovieBudget['Profitability'] = MovieBudget['Worldwide Gross'] + MovieBudget['Domestic Gross'] - MovieBudget['Production Budget']
MovieBudget['Revenue Budget Ratio'] = (MovieBudget['Worldwide Gross'] + MovieBudget['Domestic Gross']) / MovieBudget['Production Budget']

# MovieBudget.head()

In [8]:
# filtered movies df by date and budget 

Movies_After_Date = 1990
Budget_Max = 5000000
Budget_Min = 0

MovieBudgetFiltered = MovieBudget[(MovieBudget['Production Budget'] < Budget_Max) & (MovieBudget['Production Budget'] > Budget_Min)]
MovieBudgetFiltered.sort_values('Revenue Budget Ratio', ascending=False, inplace=True)
MovieBudgetFiltered.reset_index(drop=True, inplace=True)
MovieBudgetFiltered[:50]
MovieBudgetFiltered['Release Date'] = MovieBudgetFiltered['Release Date'].str[-4:].astype(int)
MovieBudgetFiltered= MovieBudgetFiltered[(MovieBudgetFiltered['Release Date']>Movies_After_Date)].reset_index(drop=True)
MovieBudgetFiltered['Movie'] = MovieBudgetFiltered['Movie'].str.replace('â', "'")
MovieBudgetFiltered

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_952/3256675032.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MovieBudgetFiltered.sort_values('Revenue Budget Ratio', ascending=False, inplace=True)
/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_952/3256675032.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MovieBudgetFiltered['Release Date'] = MovieBudgetFiltered['Release Date'].str[-4:].astype(int)


,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,Revenue Budget Ratio
0,93,2009,Paranormal Activity,450000,107918810,194183034,301651844,671.337431
1,7,1999,The Blair Witch Project,600000,140539099,248300000,388239099,648.065165
2,80,2015,The Gallows,100000,22764410,41656474,64320884,644.208840
3,74,1993,El Mariachi,7000,2040920,2041928,4075848,583.264000
4,10,2004,Super Size Me,65000,11529368,22233808,33698176,519.433477
...,...,...,...,...,...,...,...,...
1131,28,2011,2:13,3500000,0,0,-3500000,0.000000
1132,29,2013,"Batman: The Dark Knight Returns, Part 2",3500000,0,0,-3500000,0.000000
1133,30,2014,In the Name of the King III,3500000,0,0,-3500000,0.000000
1134,31,2014,Wicked Blood,3500000,0,0,-3500000,0.000000


In [9]:
# Connecting to Movie Basics Table where genres reside

movie_basics_query = """ 
    SELECT *
    FROM movie_basics
"""

movie_basics_df = pd.read_sql(movie_basics_query, conn)
clean_column_headers(movie_basics_df)
movie_basics_df['Genres'] = clean_comma_series(movie_basics_df['Genres'])
movie_basics_df
movie_genres_df = pd.DataFrame([movie_basics_df['Primary Title'], movie_basics_df['Genres']])
movie_genres_df = movie_genres_df.transpose()
movie_genres_df.columns = movie_genres_df.columns.str.replace('Primary Title', 'Movie')
movie_genres_df = movie_genres_df.drop_duplicates('Movie')
movie_genres_df

,Movie,Genres
0,Sunghursh,"[Action, Crime, Drama]"
1,One Day Before the Rainy Season,"[Biography, Drama]"
2,The Other Side of the Wind,[Drama]
3,Sabse Bada Sukh,"[Comedy, Drama]"
4,The Wandering Soap Opera,"[Comedy, Drama, Fantasy]"
...,...,...
146139,Kuambil Lagi Hatiku,[Drama]
146140,Rodolpho Teóphilo - O Legado de um Pioneiro,[Documentary]
146141,Dankyavar Danka,[Comedy]
146142,6 Gunn,None


In [10]:
# Connecting to Persons Schema and Joining with Known For and Movie Basics Schemas

persons_query = """ 
    SELECT *
    FROM persons
    JOIN directors
        USING(person_id)
    JOIN movie_basics
        USING(movie_id)
"""

joined_df = pd.read_sql(persons_query, conn)

clean_column_headers(joined_df)
joined_df = pd.DataFrame(data=joined_df, columns=['Primary Name', 'Primary Title', 'Primary Profession'])
# joined_df['Primary Profession'] = clean_comma_series(joined_df['Primary Profession'])
# joined_df.columns = joined_df.columns.str.replace('Primary Profession', 'Profession')
joined_df.columns = joined_df.columns.str.replace('Primary Title', 'Movie')
joined_df.columns = joined_df.columns.str.replace('Primary Name', 'Name')
joined_df = joined_df.drop_duplicates()

joined_df

,Name,Movie,Primary Profession
0,Ruel S. Bayani,Paano na kaya,"director,production_manager,miscellaneous"
4,Ruel S. Bayani,No Other Woman,"director,production_manager,miscellaneous"
7,Ruel S. Bayani,One More Try,"director,production_manager,miscellaneous"
8,Ruel S. Bayani,Kasal,"director,production_manager,miscellaneous"
10,Bryan Beasley,The Quiet Philanthropist: The Edith Gaylord Story,"director,producer,writer"
...,...,...,...
291164,Zheng Wei,The Old Road,director
291165,Rama Narayanan,Chain Jayapal,"director,writer"
291167,Rama Narayanan,Arya Suriya,"director,writer"
291168,Samir Eshra,The Shadow Lawyers,"director,writer,cinematographer"


In [11]:
# merged data sets

Merged_df = MovieBudgetFiltered.merge(movie_genres_df, on = 'Movie', how='left')
Merged_df = Merged_df.merge(joined_df, on = 'Movie', how='left')

Merged_df

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,Revenue Budget Ratio,Genres,Name,Primary Profession
0,93,2009,Paranormal Activity,450000,107918810,194183034,301651844,671.337431,NaN,NaN,NaN
1,7,1999,The Blair Witch Project,600000,140539099,248300000,388239099,648.065165,NaN,NaN,NaN
2,80,2015,The Gallows,100000,22764410,41656474,64320884,644.208840,"[Horror, Mystery, Thriller]",Chris Lofing,"producer,director,writer"
3,80,2015,The Gallows,100000,22764410,41656474,64320884,644.208840,"[Horror, Mystery, Thriller]",Travis Cluff,"producer,actor,director"
4,74,1993,El Mariachi,7000,2040920,2041928,4075848,583.264000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1573,28,2011,2:13,3500000,0,0,-3500000,0.000000,NaN,NaN,NaN
1574,29,2013,"Batman: The Dark Knight Returns, Part 2",3500000,0,0,-3500000,0.000000,NaN,NaN,NaN
1575,30,2014,In the Name of the King III,3500000,0,0,-3500000,0.000000,NaN,NaN,NaN
1576,31,2014,Wicked Blood,3500000,0,0,-3500000,0.000000,NaN,NaN,NaN


In [12]:
# Explode Datasets

MergedExploded_df = Merged_df.explode('Genres')
MergedExploded_df.dropna(inplace=True)
MergedExploded_df.columns = MergedExploded_df.columns.str.replace('Revenue Budget Ratio', 'RBR')

# MergedExploded_df.dropna(subset='Genres', inplace=True)

MergedExploded_df


,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,RBR,Genres,Name,Primary Profession
2,80,2015,The Gallows,100000,22764410,41656474,64320884,644.20884,Horror,Chris Lofing,"producer,director,writer"
2,80,2015,The Gallows,100000,22764410,41656474,64320884,644.20884,Mystery,Chris Lofing,"producer,director,writer"
2,80,2015,The Gallows,100000,22764410,41656474,64320884,644.20884,Thriller,Chris Lofing,"producer,director,writer"
3,80,2015,The Gallows,100000,22764410,41656474,64320884,644.20884,Horror,Travis Cluff,"producer,actor,director"
3,80,2015,The Gallows,100000,22764410,41656474,64320884,644.20884,Mystery,Travis Cluff,"producer,actor,director"
...,...,...,...,...,...,...,...,...,...,...,...
1569,27,2006,The Secret,3500000,0,0,-3500000,0.00000,Documentary,Barbara Wong Chun-Chun,"director,writer,actress"
1570,27,2006,The Secret,3500000,0,0,-3500000,0.00000,Documentary,Kaf,"director,animation_department"
1571,27,2006,The Secret,3500000,0,0,-3500000,0.00000,Documentary,Cyop,"director,animation_department"
1572,27,2006,The Secret,3500000,0,0,-3500000,0.00000,Documentary,Ben Wicks,"director,assistant_director,writer"


In [13]:
MergedExploded_df['Genres'].value_counts()

Drama          515
Thriller       208
Horror         167
Comedy         163
Documentary    146
Romance        115
Mystery         86
Action          82
Crime           76
Sci-Fi          57
Adventure       38
Family          33
Biography       25
Fantasy         22
History         22
Music           20
Sport           19
Western         10
Musical          5
War              5
Animation        3
Name: Genres, dtype: int64

In [14]:
# Mean RBR by Genre
MergedExploded_df.groupby(by='Genres').mean()['RBR'].sort_values(ascending=False)

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_952/352577621.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  MergedExploded_df.groupby(by='Genres').mean()['RBR'].sort_values(ascending=False)


Genres
Mystery        21.873141
Horror         16.070580
Thriller       11.460368
Fantasy         7.922350
Drama           7.514172
Romance         6.759905
Biography       5.793482
Comedy          3.926225
Music           3.183305
Sci-Fi          3.163682
Documentary     2.936084
Crime           2.848469
Western         2.800075
Action          2.484872
Adventure       2.384197
Family          2.077625
Sport           1.493817
Musical         0.948882
War             0.759267
History         0.747271
Animation       0.000000
Name: RBR, dtype: float64

In [20]:
# Linear Regression and Anova
res = smf.ols(formula="RBR ~ C(Genres)", data=MergedExploded_df).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RBR   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.544
Date:                Wed, 02 Aug 2023   Prob (F-statistic):             0.0582
Time:                        16:51:22   Log-Likelihood:                -9246.9
No. Observations:                1817   AIC:                         1.854e+04
Df Residuals:                    1796   BIC:                         1.865e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    2.4849      4.361      0.570      0.569      -6.068      11.037
C(Genres)[T.Adventure]      -0.1007      7.749     -0.013      0.990     -15.299      15.098
C(Genres)[T.Animation]      -2.4849     23.211     -0.107      0.915     -48.009      43.039
C(Genres)[T.Biography]       3.3086      9.021      0.367      0.714     -14.385      21.002
C(Genres)[T.Comedy]          1.4414      5.346      0.270      0.787      -9.044      11.927
C(Genres)[T.Crime]           0.3636      6.287      0.058      0.954     -11.968      12.695
C(Genres)[T.Documentary]     0.4512      5.449      0.083      0.934     -10.237      11.139
C(Genres)[T.Drama]           5.0293      4.695      1.071      0.284      -4.179      14.238
C(Genres)[T.Family]         -0.4072      8.140     -0.050      0.960     -16.373      15.558
C(Genres)[T.Fantasy]         5.4375      9.481      0.574      0.566     -13.158      24.033
C(Genres)[T.History]        -1.7376      9.481     -0.183      0.855     -20.333      16.858
C(Genres)[T.Horror]         13.5857      5.325      2.551      0.011       3.142      24.029
C(Genres)[T.Music]           0.6984      9.848      0.071      0.943     -18.616      20.013
C(Genres)[T.Musical]        -1.5360     18.190     -0.084      0.933     -37.211      34.139
C(Genres)[T.Mystery]        19.3883      6.095      3.181      0.001       7.435      31.342
C(Genres)[T.Romance]         4.2750      5.707      0.749      0.454      -6.919      15.469
C(Genres)[T.Sci-Fi]          0.6788      6.810      0.100      0.921     -12.677      14.034
C(Genres)[T.Sport]          -0.9911     10.054     -0.099      0.921     -20.710      18.728
C(Genres)[T.Thriller]        8.9755      5.149      1.743      0.081      -1.123      19.074
C(Genres)[T.War]            -1.7256     18.190     -0.095      0.924     -37.401      33.950
C(Genres)[T.Western]         0.3152     13.227      0.024      0.981     -25.626      26.256
==============================================================================
Omnibus:                     3377.734   Durbin-Watson:                   0.114
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3503994.371
Skew:                          13.701   Prob(JB):                         0.00
Kurtosis:                     216.382   Cond. No.                         28.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
TopThreeGenres_df = MergedExploded_df.loc[(MergedExploded_df['Genres'] == 'Horror') | (MergedExploded_df['Genres'] == 'Thriller') | (MergedExploded_df['Genres'] == 'Mystery')].reset_index(drop=True)

In [17]:
TopThreeGenres_df.drop_duplicates('Id', inplace=True)

In [18]:
TopThreeGenres_df.groupby(by='Name').mean()['RBR'].sort_values(ascending=False)[:50]

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_952/2006494504.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  TopThreeGenres_df.groupby(by='Name').mean()['RBR'].sort_values(ascending=False)[:50]


Name
Chris Lofing             644.208840
William Brent Bell       155.022435
James Wan                102.586691
Levan Gabriadze           97.153843
Tod Williams              87.421646
Bradley Parker            60.531361
James DeMonaco            51.913232
Adam Wingard              45.381183
Scott Derrickson          45.271570
Jeff Wadlow               39.010960
Daniel Stamm              30.896044
Stephen Susco             25.301333
Rob Cohen                 22.352001
Deon Taylor               20.215716
Marcus Nispel             19.591680
David Robert Mitchell     18.962416
Laura Lau                 14.675249
Steven Soderbergh         14.623317
Gareth Edwards            11.754062
Eli Roth                  11.653068
Debra Granik              11.331527
Rodrigo Cortés            11.226540
Ed Gass-Donnelly          10.157003
Chris Pickle               9.991363
Leigh Whannell             9.510138
Sean Durkin                8.419949
J.C. Chandor               7.584357
William Eubank         

In [19]:
TopThreeGenres_df['Name'].value_counts()

Rob Zombie        2
Daniel Stamm      2
William Eubank    2
Kevin Smith       2
Eli Roth          2
                 ..
Zal Batmanglij    1
J.C. Chandor      1
Sean Durkin       1
Leigh Whannell    1
Travis Romero     1
Name: Name, Length: 82, dtype: int64